In [31]:
import pertpy as pt
import scanpy as sc
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300
import seaborn as sns
from scipy import stats

from statsmodels.sandbox.stats.multicomp import multipletests
from sparsecca import lp_pmd, multicca_permute, multicca_pmd


In [2]:
%load_ext autoreload
%autoreload 2

import importlib
importlib.reload(pt)

Installed version 0.4.0 of pertpy is newer than the latest release 0.3.0! You are running a nightly version and 
features may break!

<module 'pertpy' from '/Users/tessa/miniforge3/envs/pertpy5/lib/python3.9/site-packages/pertpy/__init__.py'>

In [3]:
adata = sc.read_h5ad("zhang_with_PCs.h5ad")
adata = adata[adata.obs['Origin']=="t" ,:].copy()
adata = adata[adata.obs['Sample'].str.contains('Pre'),:].copy()
isecs = pd.crosstab(adata.obs['Cluster'], adata.obs['Sample'])
celltypes = isecs[(isecs > 3).sum(axis=1) >= np.shape(isecs)[1]-2].index.values.to_list()
adata = adata[adata.obs['Cluster'].isin(celltypes),:].copy()

In [4]:
isecs = pd.crosstab(adata.obs['Cluster'], adata.obs['Sample'])
keep_pts = isecs.loc[:,(isecs > 3).sum(axis=0) == isecs.shape[0]].columns.values.to_list()
adata = adata[adata.obs['Sample'].isin(keep_pts),:].copy()

In [5]:
sc.pp.neighbors(adata)
sc.tl.umap(adata)

In [6]:
dl = pt.tl.Dialogue(sample_id = "Sample",
                   celltype_key = "Cluster",
                   n_counts_key = "Number of counts",
                   n_mpcs = 3)

In [16]:
agg_pca = True
solver = "bs"
normalize = True
ct_order = None

In [18]:
if ct_order is not None:
            cell_types = ct_order
else:
            ct_order = cell_types = adata.obs[dl.celltype_key].astype("category").cat.categories

In [19]:
ct_order

Index(['t_Bmem-CD27', 't_CD4_Tcm-LMNA', 't_CD4_Treg-FOXP3', 't_CD8_MAIT-KLRB1',
       't_CD8_Tem-GZMK', 't_CD8_Trm-ZNF683', 't_Tn-LEF1', 't_mono-FCN1',
       't_pB-IGHG1'],
      dtype='object')

In [21]:
mcca_in, ct_subs = dl.load(adata, ct_order=cell_types, agg_pca=agg_pca, normalize=normalize)


In [28]:
np.shape(mcca_in)
# 9 cell types, 11 patients, 50 PCs

(9, 11, 50)

In [33]:
n_samples = mcca_in[0].shape[1]

In [36]:
penalties = multicca_permute(
                mcca_in, penalties=np.sqrt(n_samples) / 2, nperms=10, niter=50, standardize=True
            )["bestpenalties"]

In [35]:
n_samples

50

In [38]:
multicca_permute(
                mcca_in, penalties=np.sqrt(n_samples) / 2, nperms=10, niter=50, standardize=True
            )

{'pvals': array([0.]),
 'zstat': array([5.02866585]),
 'bestpenalties': array([3.53553391, 3.53553391, 3.53553391, 3.53553391, 3.53553391,
        3.53553391, 3.53553391, 3.53553391, 3.53553391]),
 'cors': array([35.06807004]),
 'penalties': array([[3.53553391],
        [3.53553391],
        [3.53553391],
        [3.53553391],
        [3.53553391],
        [3.53553391],
        [3.53553391],
        [3.53553391],
        [3.53553391]]),
 'nperms': 10}

In [40]:
ws, _ = multicca_pmd(mcca_in, penalties, K=dl.n_mcps, standardize=True, niter=100, mimic_R=normalize)

In [41]:
ws

[array([[ 0.        ,  0.14883359,  0.        ],
        [-0.        ,  0.3713363 ,  0.        ],
        [-0.        , -0.        , -0.07731346],
        [-0.        ,  0.33364791, -0.        ],
        [ 0.        , -0.43418763,  0.        ],
        [ 0.        , -0.4149118 ,  0.        ],
        [-0.        ,  0.03415326,  0.        ],
        [ 0.        ,  0.        ,  0.31847857],
        [-0.        ,  0.21902881, -0.        ],
        [ 0.        , -0.03262751,  0.        ],
        [ 0.        ,  0.19374271,  0.        ],
        [ 0.        , -0.        ,  0.        ],
        [-0.22009683, -0.        , -0.        ],
        [-0.        ,  0.        , -0.16475454],
        [-0.26720495,  0.        , -0.01140822],
        [ 0.05783899,  0.        ,  0.        ],
        [-0.10305554,  0.02066696, -0.00688652],
        [-0.        ,  0.        ,  0.03408164],
        [ 0.        , -0.        ,  0.        ],
        [ 0.08623481, -0.19641499,  0.35398328],
        [ 0.3874245 

In [42]:
ws_dict = {ct: ws[i] for i, ct in enumerate(ct_order)}

In [43]:
pre_r_scores = {
            ct: ct_subs[ct].obsm["X_pca"][:, :50] @ ws[i] for i, ct in enumerate(cell_types)  # TODO change from 50
        }

In [46]:
mcp_scores = {
            ct: dl._get_residuals(ct_subs[ct].obs[dl.n_counts_key].values, pre_r_scores[ct].T).T
            for i, ct in enumerate(cell_types)
        }

ValueError: could not convert string to float: '1,068 '

In [50]:
ct_subs[cell_types[0]].obs["Number of counts"]

Cell barcode
AAATGCCTCATCTGCC.Pre_P007_t    1,068 
AATCCAGAGCGTTCCG.Pre_P007_t    1,309 
ACAGCCGAGCACCGCT.Pre_P007_t    1,344 
ACCGTAAAGCTAACTC.Pre_P007_t      992 
ACGAGCCTCGTAGGTT.Pre_P007_t    1,482 
                                ...  
CTGATAGTCAGGCAAG.Pre_P004_t    3,181 
CTTAACTCAGTAAGCG.Pre_P004_t    1,860 
GCACATATCTGTTTGT.Pre_P004_t    2,736 
TCATTTGCACGCTTTC.Pre_P004_t    2,636 
TCGTAGATCCCTAATT.Pre_P004_t    4,360 
Name: Number of counts, Length: 7283, dtype: category
Categories (3850, object): ['1,000 ', '1,001 ', '1,002 ', '1,003 ', ..., '995 ', '996 ', '997 ', '998 ']